In [ ]:
from __future__ import division, print_function
import os
import logging
import glob

# Third-party
from astropy import log as logger
from astropy.constants import G
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
import h5py
import matplotlib.animation as animation
from astropy.stats import median_absolute_deviation

from astroML.plotting.tools import draw_ellipse

# Custom
import gary.coordinates as gc
import gary.dynamics as gd
from gary.dynamics import mockstream
import gary.integrate as gi
import gary.potential as gp
from gary.units import galactic, UnitSystem

from scf import SCFSimulation

In [ ]:
rs = 10.*u.kpc
M = ((220.*u.km/u.s)**2 * rs / G).to(u.Msun)
potential = gp.IsochronePotential(m=M, b=rs, units=galactic)

In [ ]:
with h5py.File("/Users/adrian/projects/talks/thesis_colloquium/snap.h5", 'r') as f:
    print(list(f.keys()))
    
    usys = UnitSystem([u.Unit(x) for x in f['units'].attrs.values()] + [u.radian])
    for i in range(3):
        print((f['snapshots/{}'.format(i)].attrs['t'] * usys['time']).to(u.Myr))
        
    sim_dt = f['parameters'].attrs['dt']
    dt = (sim_dt*usys['time']).to(u.Myr).value
    pos = (f['snapshots/2/pos'][:]*usys['length']).to(u.kpc).value
    vel = (f['snapshots/2/vel'][:]*usys['length']/usys['time']).to(u.km/u.s).value
    tub = (f['snapshots/2/tub'][:]*usys['time']).to(u.Myr).value

    cen_pos = (f['cen/pos'][:]*usys['length']).to(u.kpc).value
    cen_vel = (f['cen/vel'][:]*usys['length']/usys['time']).to(u.km/u.s).value
    cen_t = np.arange(cen_vel.shape[1])*sim_dt

In [ ]:
f77_path = "/Users/adrian/projects/talks/thesis_colloquium/fortran/"
f77_cen = np.genfromtxt(os.path.join(f77_path, "SCFCEN"), names=['t','derp','x','y','z','vx','vy','vz'])
for n in 'xyz':
    f77_cen[n] = (f77_cen[n]*usys['length']).to(u.kpc).value
    f77_cen['v'+n] = (f77_cen['v'+n]*usys['length']/usys['time']).to(u.km/u.s).value

In [ ]:
pl.plot(f77_cen['x'], f77_cen['y'])
pl.plot(cen_pos[0], cen_pos[1])

---

In [ ]:
snapfiles = glob.glob(os.path.join(f77_path,"SNAP*"))

f77_tub = np.zeros((len(snapfiles),pos.shape[-1]))
f77_pos = np.zeros((len(snapfiles),3,pos.shape[-1]))
f77_vel = np.zeros((len(snapfiles),3,pos.shape[-1]))

for i,snapfile in enumerate(snapfiles):
    d = np.loadtxt(snapfile, usecols=[1,2,3,4,5,6,9], skiprows=1)
    f77_pos[i] = d[:,:3].T
    f77_vel[i] = d[:,3:6].T
    f77_tub[i] = d[:,6]
    
f77_pos = (f77_pos*usys['length']).to(u.kpc)
f77_vel = (f77_vel*usys['length']/usys['time']).to(u.kpc/u.Myr)

In [ ]:
f77_w = gd.CartesianPhaseSpacePosition(f77_pos[0], f77_vel[0])

In [ ]:
actions,angles,freqs = potential.action_angle(f77_w)

In [ ]:
pl.plot(freqs[0], freqs[1], ls='none')